## Import statements

In [1]:
# Import Statements

from financetoolkit import Toolkit
import yfinance as yf
import pandas as pd
from pathlib import Path

In [2]:
api_key = 'DhSKlDzkmluzGeZ4mK9mvI0lQDUDmRpa'

## Sample financetoolkit syntax

In [3]:
# Sample

# companies = Toolkit(
#     tickers=['ACLS'],
#     api_key=api_key,
# )

# balance_sheet = companies.get_balance_sheet_statement()
# income_statement = companies.get_income_statement()
# # cash_flow_statement = companies.get_cash_flow_statement()

In [4]:
# balance_sheet.head(50)

In [5]:
# income_statement.head(50)

## Initialize a DataFrame for the stock data

Note: It won't be necessary to initialize the dataframe, because concatenating to empty dataframes will be deprecated in a future pandas version.  However, the initial code will be left below and commented out.

In [6]:
# Initialize a DataFrame for the stock data
# stock_df = pd.DataFrame(columns=[
#     'year',
#     'gross_profit_margin',
#     'ebit',
#     'return_on_sales',
#     'return_on_assets',
#     'return_on_equity',
#     'ebitda',
#     'eps'
# ])

In [7]:
# stock_df

## Create a list of mid-cap ticker symbols

In [8]:
list_of_mid_cap_tickers = [
'BYDIY',
'ONTO',
'COHR',
'BYDIF',
'OLED',
'MKSI',
'OMRNY',
'AMKR',
'SOTGY',
'FN',
'SNPTF',
'CGNX',
'VNT',
'LFUS',
'SUMCF',
'NOVT',
# 'HPHTF', # No data available for this one
'SUOPY',
'SLOIF',
'SEKEY',
'ST',
'CLS',
'NVMI',
'ASMVY',
'SEPJF',
'BMI',
'AUOTY',
'HRIBF',
'ITRI',
'AIXXF',
'IPGP',
'RNSHF',
'HROEY',
'CAMT',
'ACLS',
'SHCAY',
'SANM',
'LPL',
'VEMLY',
'TYOYY',
'KLIC',
'ESE',
'PLXS',
'IONQ',
'AMBA',
'SONO',
'OSIS',
'VZIO',
'ROG',
'KGBLY'
]

## YFinance Approach

Note: We switched from using YFinance to financetoolkit instead.

In [9]:
# # # Yahoo Finance Approach
# # Create a loop for all of the ticker symbols
# for ticker_symbol in list_of_mid_cap_tickers:
#     # For each ticker, grab the data from yfinance
#     stock = yf.Ticker(ticker_symbol)
#     balance_sheet = stock.get_balance_sheet()
#     income_statement = stock.get_income_stmt()
    
#     # # Grab or derive the parameters that we're interested in
#     # Gross_Profit_Margin = Gross_Profit -
#     gross_profit_margin = income_statement.loc['GrossProfit']['2023-12-31']
    
#     # EBIT comes from the income statement
#     ebit = income_statement.loc['EBIT']['2023-12-31']
#     # Return_on_Sales = EBIT / Total_Revenue
#     return_on_sales = ebit / ( income_statement.loc['TotalRevenue']['2023-12-31'] )
#     # Grab the Net_Income
#     net_income = income_statement.loc['NetIncome']['2023-12-31']
#     # Return_on_Assets = Net_Income / Total_Assets
#     return_on_assets = net_income / ( balance_sheet.loc['TotalAssets']['2023-12-31'] )
#     # Return_on_Equity = Net_Income / Shareholder_Equity
#     return_on_equity = net_income / ( balance_sheet.loc['StockholdersEquity']['2023-12-31'] )
#     # EBITDA comes from the income statement
#     ebitda = income_statement.loc['EBITDA']['2023-12-31']
#     # Basic EPS comes from the incmoe statement
#     basic_eps = income_statement.loc['BasicEPS']['2023-12-31']

## Financetoolkit approach

In [10]:
# Delete the stock_df if it exists
# These lines are only necessary when the code is being re-run for some reason
try:
    del mid_cap_stock_df
except:
    pass

In [11]:
# # FinanceToolkit Approach

# Create a loop for all of the ticker symbols
for ticker_symbol in list_of_mid_cap_tickers:
    
    # For each ticker, grab the data from FinanceToolkit
    stock = Toolkit(
        tickers=[ticker_symbol],
        api_key=api_key,
    )
    balance_sheet = stock.get_balance_sheet_statement()
    income_statement = stock.get_income_statement()

    # Grab the most recent year of available data for the stock
    year = max(balance_sheet.columns)
    
    # # Grab or derive the parameters that we're interested in
    # Gross_Profit_Margin = Gross_Profit - Cost of Goods Sold
    gross_profit_margin = income_statement.loc['Gross Profit'][year] - income_statement.loc['Cost of Goods Sold'][year]
    # EBIT comes from the income statement
    ebit = income_statement.loc['EBITDA'][year] + income_statement.loc['Depreciation and Amortization'][year]
    # Return_on_Sales = EBIT / Total_Revenue
    return_on_sales = ebit / ( income_statement.loc['Revenue'][year] )
    # Grab the Net_Income
    net_income = income_statement.loc['Net Income'][year]
    # Return_on_Assets = Net_Income / Total_Assets
    return_on_assets = net_income / ( balance_sheet.loc['Total Assets'][year] )
    # Return_on_Equity = Net_Income / Shareholder_Equity
    return_on_equity = net_income / ( balance_sheet.loc['Total Shareholder Equity'][year] )
    # EBITDA comes from the income statement
    ebitda = income_statement.loc['EBITDA'][year]
    # Basic EPS comes from the incmoe statement
    eps = income_statement.loc['EPS'][year]
    # Avg_Revenue_Growth_3_yrs = ( ( Current_Year_Revenue / 3_Years_Ago_Revenue ) ** (1/3) ) - 1
    # avg_revenue_growth_3_yrs = ( ( income_statement.loc['Revenue'][year] / income_statement.loc['Revenue'][str( int(year.strftime('%Y')) - 3 )] ) ** (1/3) ) - 1
    avg_revenue_growth_1_yr = ( ( income_statement.loc['Revenue'][year] / income_statement.loc['Revenue'][str( int(year.strftime('%Y')) - 1 )] ) ** (1/1) ) - 1
    # Debt_to_Sales = Total_Debt / Revenue
    debt_to_sales = balance_sheet.loc['Total Debt'][year] / income_statement.loc['Revenue'][year]

    # Add the values to a DataFrame
    temp_df = pd.DataFrame(
        {
        'year': year,
        'gross_profit_margin': gross_profit_margin,
        'ebit': ebit,
        'return_on_sales': return_on_sales,
        'return_on_assets': return_on_assets,
        'return_on_equity': return_on_equity,
        'ebitda': ebitda,
        'eps': eps,
        # 'avg_revenue_growth_3_yrs': avg_revenue_growth_3_yrs,
        'avg_revenue_growth_1_yr': avg_revenue_growth_1_yr,
        'debt_to_sales': debt_to_sales
        },
    index=[ticker_symbol]
    )
    
    # Concatenate the temporary dataframe to the one that holds the rest of the stock values
    # Check to see if the stock_df already exists
    try:
        mid_cap_stock_df
    # If the stock dataframe doesn't exist yet, initialize it
    except:
        mid_cap_stock_df = temp_df
    # If the stock dataframe already exists, then concatenate it with the temporary dataframe
    else:
        mid_cap_stock_df = pd.concat([mid_cap_stock_df, temp_df], axis="rows")

Obtaining income data: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.61it/s]


In [12]:
# Give a column name to the index
mid_cap_stock_df.index.name = "ticker"

In [13]:
# Check the stock dataframe
mid_cap_stock_df.head()

,year,gross_profit_margin,ebit,return_on_sales,return_on_assets,return_on_equity,ebitda,eps,avg_revenue_growth_1_yr,debt_to_sales
ticker,,,,,,,,,,
BYDIY,2022,-94485752000.0,7854485000.0,0.07327882275389554,0.03259300703246617,0.07246393602451393,4937601000.0,41.0,0.2035697865247572,0.02732837431593862
ONTO,2023,-70950000.0,261100000.0,0.3200272593115553,0.06344359777809429,0.06977054882279943,193888000.0,2.47,-0.1883388397933511,0.02392666460751004
COHR,2023,-1923534000.0,732102000.0,0.1418774829945156,-0.018923162659132547,-0.05202112219002873,50415000.0,-1.89,0.5558328127223653,0.8699089164938664
BYDIF,2022,-94485752000.0,7854485000.0,0.07327882275389554,0.03259300703246617,0.07246393602451393,4937601000.0,0.82,0.2035697865247572,0.02732837431593862
OLED,2023,305677000.0,303993000.0,0.5273728421019761,0.12163915154398455,0.14027594860788847,260591000.0,0.0043,-0.06517801105707088,0.03964928898441959


In [14]:
# balance_sheet.head(50)

In [15]:
# income_statement.head(50)

## Save the mid_cap_stock_df

The dataframe will be saved to a csv file, so that multiple API calls won't be necessary when the model is built.

In [16]:
# Save the stock_df to a csv, so that multiple API calls aren't necessary to make the model
filename = 'mid_cap_stock_data'

mid_cap_stock_df.to_csv(Path(f'./Data_Files/{filename}.csv'))

## Create a list of ticker symbols that were successfully acquired

Note that these companies had to be publicly traded at the time of acquisition, otherwise the financial data won't be available through the API

In [17]:
list_of_successful_acquisition_tickers = [
'WORK', # Slack, acquired by Salesforce in 2021
'FIT', # Fitbit, acquired by Google in 2019
'LNKD', # Linkedin, acquired by Microsoft in 2016, only two years worth of data
'ATVI', # Activision, acquired by Microsoft in 2023
# 'BRCM', # Broadcom, acquired by Avago Technologies in 2016, only one year worth of data (two needed)
'ARM', # ARM Holdings, almost acquired by Nvidia, but stopped by regulatory concerns
# # Ticker symbols listed below aren't usable
# 'MBLY', # Mobileye, acquired by Intel, was taken private in 2017, went public again in 2022, but the API only has 5 years worth of data in the balance sheet
#'SUNW', # Sun Microsystems, acquired by Oracle in 2010, but the ticker has been reissued to a different company
# 'APPD', AppDynamics, acquired by Cisco in 2017, but the data wasn't available through the API
]

## Financetoolkit approach

In [18]:
# Delete the stock_df if it exists
# These lines are only necessary when the code is being re-run for some reason
try:
    del successful_acquisition_stock_df
except:
    pass

In [19]:
# # FinanceToolkit Approach

# Create a loop for all of the ticker symbols
for ticker_symbol in list_of_successful_acquisition_tickers:
    
    # For each ticker, grab the data from FinanceToolkit
    stock = Toolkit(
        tickers=[ticker_symbol],
        api_key=api_key,
    )
    balance_sheet = stock.get_balance_sheet_statement()
    income_statement = stock.get_income_statement()

    # Grab the most recent year of available data for the stock
    # if ticker_symbol == 'ARM':
    #     year = 2020
    # else:
    #     year = max(balance_sheet.columns)
    year = max(balance_sheet.columns)
    
    # # Grab or derive the parameters that we're interested in
    # Gross_Profit_Margin = Gross_Profit - Cost of Goods Sold
    gross_profit_margin = income_statement.loc['Gross Profit'][year] - income_statement.loc['Cost of Goods Sold'][year]
    # EBIT comes from the income statement
    ebit = income_statement.loc['EBITDA'][year] + income_statement.loc['Depreciation and Amortization'][year]
    # Return_on_Sales = EBIT / Total_Revenue
    return_on_sales = ebit / ( income_statement.loc['Revenue'][year] )
    # Grab the Net_Income
    net_income = income_statement.loc['Net Income'][year]
    # Return_on_Assets = Net_Income / Total_Assets
    return_on_assets = net_income / ( balance_sheet.loc['Total Assets'][year] )
    # Return_on_Equity = Net_Income / Shareholder_Equity
    return_on_equity = net_income / ( balance_sheet.loc['Total Shareholder Equity'][year] )
    # EBITDA comes from the income statement
    ebitda = income_statement.loc['EBITDA'][year]
    # Basic EPS comes from the income statement
    eps = income_statement.loc['EPS'][year]
    # Avg_Revenue_Growth_3_yrs = ( ( Current_Year_Revenue / 3_Years_Ago_Revenue ) ** (1/3) ) - 1
    # avg_revenue_growth_3_yrs = ( ( income_statement.loc['Revenue'][year] / income_statement.loc['Revenue'][str( int(year.strftime('%Y')) - 3 )] ) ** (1/3) ) - 1
    avg_revenue_growth_1_yr = ( ( income_statement.loc['Revenue'][year] / income_statement.loc['Revenue'][str( int(year.strftime('%Y')) - 1 )] ) ** (1/1) ) - 1
    # Debt_to_Sales = Total_Debt / Revenue
    debt_to_sales = balance_sheet.loc['Total Debt'][year] / income_statement.loc['Revenue'][year]
    
    # Add the values to a DataFrame
    temp_df = pd.DataFrame(
        {
        'year': year,
        'gross_profit_margin': gross_profit_margin,
        'ebit': ebit,
        'return_on_sales': return_on_sales,
        'return_on_assets': return_on_assets,
        'return_on_equity': return_on_equity,
        'ebitda': ebitda,
        'eps': eps,
        # 'avg_revenue_growth_3_yrs': avg_revenue_growth_3_yrs,
        'avg_revenue_growth_1_yr': avg_revenue_growth_1_yr,
        'debt_to_sales': debt_to_sales
        },
    index=[ticker_symbol]
    )
    
    # Concatenate the temporary dataframe to the one that holds the rest of the stock values
    # Check to see if the stock_df already exists
    try:
        successful_acquisition_stock_df
    # If the stock dataframe doesn't exist yet, initialize it
    except:
        successful_acquisition_stock_df = temp_df
    # If the stock dataframe already exists, then concatenate it with the temporary dataframe
    else:
        successful_acquisition_stock_df = pd.concat([successful_acquisition_stock_df, temp_df], axis="rows")

Obtaining income data: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.48it/s]


In [20]:
# Give a column name to the index
successful_acquisition_stock_df.index.name = "ticker"

In [21]:
# Check the stock dataframe
successful_acquisition_stock_df.head()

,year,gross_profit_margin,ebit,return_on_sales,return_on_assets,return_on_equity,ebitda,eps,avg_revenue_growth_1_yr,debt_to_sales
ticker,,,,,,,,,,
WORK,2020,659226000.0,-204996000.0,-0.22711470070129958,-0.1234424949665119,-0.3503704628073535,-233814000.0,-0.53,0.4317552369682529,0.7603815601422541
FIT,2019,-579444000.0,-182977000.0,-0.12752894504275197,-0.23442312837058488,-0.658188698916191,-246021000.0,-1.25,-0.05105546821624318,0.06467645394302154
LNKD,2015,2153195000.0,726163000.0,0.24278990581799326,-0.023696945415470308,-0.037179967162290654,280466000.0,-1.29,0.34800589696890216,0.3766524647507064
ATVI,2022,3084000000.0,2666000000.0,0.35414452709883104,0.05525325932147683,0.07862599386790002,2168000000.0,1.92,-0.1448369873906623,0.4796758767268863
ARM,2023,2467000000.0,1025000000.0,0.3826054497946995,0.07631808913486746,0.12935077758578128,855000000.0,0.51,-0.008879023307436174,0.08174692049272117


In [27]:
# income_statement.head(50)

In [28]:
# balance_sheet.head(50)

## Save the successful_acquisition_stock_df

The dataframe will be saved to a csv file, so that multiple API calls won't be necessary when the model is built.

In [29]:
# Save the stock_df to a csv, so that multiple API calls aren't necessary to make the model
filename = 'successful_acquisition_stock_data'

successful_acquisition_stock_df.to_csv(Path(f'./Data_Files/{filename}.csv'))